In [62]:
import nbformat

# Read your notebook (assuming version 4 for example purposes)
nb = nbformat.read("ml_project.ipynb", as_version=4)

# Normalize the notebook to add missing id fields and other updates
nbformat.validator.validate(nb)

# Write the normalized notebook back to a file
nbformat.write(nb, "ml_project_normalized.ipynb")

In [63]:
%run ml_project_normalized.ipynb

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
dropped columns
cleaned intake time
cleaned intake condition
cleaned age and sex
cleaned color
cleaned breed
dropped columns


/tmp/ipykernel_890935/1199118304.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dt_series = pd.to_datetime(df['intake_time'], errors='coerce')


cleaned intake time
cleaned intake condition
cleaned age and sex
cleaned color
cleaned breed
Done running ml_project.ipynb.


In [64]:
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer, LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline
#from imblearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE, SMOTENC
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from scipy.stats import uniform

In [65]:
df_train = bucket_seasons(df_train)
df_train = time_as_cyclical(df_train)
df_train = df_train.drop(columns=['month_sin', 'month_cos'])

df_test = bucket_seasons(df_test)
df_test = time_as_cyclical(df_test)
df_test = df_test.drop(columns=['month_sin', 'month_cos'])


In [66]:
df_train = df_train.drop(columns=['size'])
df_test = df_test.drop(columns=['size'])
df_train.head()

,intake_type,intake_condition,animal_type,sex_upon_intake,age_upon_intake,breed,intake_year,primary_color,is_mix,outcome_type,season,hour_sin,hour_cos
0,Stray,Normal / Behavior,Dog,Spayed Female,96.0,2,2015,white,0,Return to Owner,Summer,1.224647e-16,-1.000000e+00
1,Stray,Normal / Behavior,Dog,Intact Male,11.0,7,2016,sable,1,Return to Owner,Spring,-1.000000e+00,-1.836970e-16
2,Public Assist,Normal / Behavior,Cat,Neutered Male,24.0,Common,2022,orange,0,Transfer,Spring,0.000000e+00,1.000000e+00
3,Owner Surrender,Normal / Behavior,Dog,Neutered Male,24.0,2,2017,chocolate,1,Return to Owner,Winter,1.224647e-16,-1.000000e+00
4,Public Assist,Normal / Behavior,Dog,Neutered Male,72.0,3,2019,black,1,Return to Owner,Spring,7.071068e-01,-7.071068e-01


In [67]:

def threshold_adjusted_cross_val(X_train, y_train, model, thresholds, skf):
    """
    Perform cross-validation with threshold adjustment after predicting probabilities.
    """
    # Store the balanced accuracy scores for each fold
    balanced_accuracy_scores = []

    for train_idx, val_idx in skf.split(X_train, y_train):
        # Split the data into training and validation sets
        X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]
        
        # Fit the model on the training fold
        model.fit(X_train_fold, y_train_fold)
        
        # Get predicted probabilities for the validation set
        probs = model.predict_proba(X_val_fold)
        
        # Adjust predictions based on the thresholds
        test_preds_adjusted = []

        for i, prob in enumerate(test_probs):
            # Get classes over their threshold
            passed = [cls for cls, p in enumerate(prob) if p >= thresholds.get(cls, 1.0)]
            if passed:
                # Pick the one with the highest probability
                best_class = max(passed, key=lambda cls: prob[cls])
            else:
                # Fallback to class with highest probability (no thresholds passed)
                best_class = np.argmax(prob)
            test_preds_adjusted.append(best_class)

        test_preds_adjusted = np.array(test_preds_adjusted)
        
        # Calculate balanced accuracy for the validation set
        balanced_acc = balanced_accuracy_score(y_val_fold, y_pred_adjusted)
        balanced_accuracy_scores.append(balanced_acc)
    
    # Return the mean balanced accuracy across all folds
    return balanced_accuracy_scores

In [68]:
# {'logreg__penalty': 'l2', 'logreg__fit_intercept': False, 'logreg__class_weight': 'balanced', 'logreg__C': 0.01}
# Best parameters: {'logreg__solver': 'lbfgs', 'logreg__penalty': 'l2', 'logreg__max_iter': 5000, 'logreg__fit_intercept': True, 'logreg__class_weight': {0: 0.4040806125462931, 1: 21.29916167664671, 2: 6.4625, 3: 1.3310979717087044, 4: 0.6360118728319565}, 'logreg__C': 0.01}

def train_logreg_classifier(X_train, y_train, X_test):
    cat_cols = ['intake_type', 'intake_condition', 'animal_type', 'sex_upon_intake', 'breed','primary_color','season', 'is_mix']
    num_cols = ['age_upon_intake', 'intake_year']    # Replace with your actual numerical columns

  #  cat_cols = [0, 1, 2, 3, 5,7,8, 10]
  #  num_cols = [4, 6] 

    all_cols = X_train.columns.tolist()

    # Convert column names to positional indices
    cat_indices = [all_cols.index(col) for col in cat_cols]
    num_indices = [all_cols.index(col) for col in num_cols]




    class_labels = np.unique(y_train)  # Get unique class labels in your target variable
    class_weights = compute_class_weight(
        class_weight='balanced',
        classes=class_labels,
        y=y_train
    )
    class_weight_dict = dict(zip(class_labels, class_weights))  # Map class labels to weights

    # Handle rare categories
    rare_threshold = 0.01
    for col in cat_cols:
        freq = X_train[col].value_counts(normalize=True)
        rare_categories = freq[freq < rare_threshold].index
        X_train[col] = X_train[col].apply(lambda x: 'Other' if x in rare_categories else x)
        X_test[col] = X_test[col].apply(lambda x: 'Other' if x in rare_categories else x)

    for col in cat_cols:
        X_train[col] = X_train[col].astype(str)
        X_test[col] = X_test[col].astype(str)


    sampling_strategy = {
        0: 55044,  # Adopted: Keep it as it is (no oversampling)
        1: 25000,  # Euth: Oversample 70% of the existing samples
        2: 10000,  # Died
        3: 30000,  # RTO
        4: 35211,  # Transfer
    }


    # Preprocessing for numerical features: Imputation (if needed) + Standardization
    numerical_transformer = Pipeline(steps=[
    #    ('imputer', SimpleImputer(strategy='median')),  # Impute missing values (if any)
        ('scaler', StandardScaler())  # Standardize numerical features
    ]) # i added mean=False just bc

    # Preprocessing for categorical features: Imputation (if needed) + One-Hot Encoding
    categorical_transformer = Pipeline(steps=[
     #   ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute missing values (if any)
        ('onehot', OneHotEncoder(handle_unknown='ignore'))  # One-Hot Encode categorical features
    ])

    # Combine both transformations into a single ColumnTransformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, num_indices),
            ('cat', categorical_transformer, cat_indices)
        ])

    # Create a pipeline that first transforms the data and then applies Logistic Regression
    pipeline = Pipeline(steps=[
      #  ('smote', SMOTENC(categorical_features=cat_cols, random_state=42, sampling_strategy=sampling_strategy)),
        ('preprocessor', preprocessor),
        ('logreg', LogisticRegression())  # You can adjust max_iter as needed
    ])

    param_distributions = {
        "logreg__C": uniform(0.01, 1), #uniform(loc=0.01, scale=10),
        "logreg__penalty": ['l2'],
        'logreg__solver' : ['lbfgs','saga'],
        "logreg__fit_intercept": [True, False],
        'logreg__class_weight': [class_weight_dict, 'balanced'],
        'logreg__max_iter' : [7000, 7500, 8000]
    }

    randomized_search = RandomizedSearchCV(
        estimator=pipeline,
        param_distributions=param_distributions,
        n_iter=10,
        cv=skf,
        scoring=balanced_acc_scorer,
        verbose=3,
        random_state=42
    )

    randomized_search.fit(X_train, y_train)

    print('Best parameters:', randomized_search.best_params_)
    print('Best cross-validation accuracy:', randomized_search.best_score_)

    # Define thresholds for each class (adjust based on your needs)
    thresholds = {
        0: 0.90,  # For Adopted
        1: 0.10,   # For Died
        2: 0.20,   # For Euth
        3: 0.30,  # For RTO
        4: 0.30    # For Transfer
    }



    cv_scores = cross_val_score(randomized_search.best_estimator_, X_train, y_train, cv=5, verbose=3, scoring=balanced_acc_scorer)
    print('Generalization Balanced accuracy (via cross_val_score):', cv_scores.mean())

    test_predictions = randomized_search.predict(X_test)
    '''
    cv_thresh_scores = threshold_adjusted_cross_val(X_train, y_train, randomized_search.best_estimator_, thresholds, skf)
    print('Generalization Balanced accuracy (via cross_val_score):', np.mean(cv_thresh_scores))

        # Adjust the test predictions using the thresholds
    test_probs = randomized_search.predict_proba(X_test)
    test_preds_adjusted = []

    for i, prob in enumerate(test_probs):
        # Get classes over their threshold
        passed = [cls for cls, p in enumerate(prob) if p >= thresholds.get(cls, 1.0)]
        if passed:
            # Pick the one with the highest probability
            best_class = max(passed, key=lambda cls: prob[cls])
        else:
            # Fallback to class with highest probability (no thresholds passed)
            best_class = np.argmax(prob)
        test_preds_adjusted.append(best_class)

    test_preds_adjusted = np.array(test_preds_adjusted)
    
    coefs = randomized_search.best_estimator_.coef_
    importance_df = pd.DataFrame(coefs.T, columns= randomized_search.best_estimator_.classes_, index= X_train.columns)
    '''

    return randomized_search.best_estimator_, test_predictions#, importance_df

In [69]:
# Thresholding function - need tadjust in CV later
def apply_thresholds(y_proba, thresholds):
    preds = []
    for prob in y_proba:
        # Pick classes where prob >= threshold
        candidates = [i for i, p in enumerate(prob) if p >= thresholds[i]]
        if candidates:
            pred = max(candidates, key=lambda i: prob[i])
        else:
            pred = np.argmax(prob)
        preds.append(pred)
    return np.array(preds)

In [70]:
X = df_train.drop(columns=['outcome_type'])
y = df_train['outcome_type']

X_train, X_test, y_train, y_test = train_test_split(X,y , test_size=0.20, random_state=42)

In [71]:
# Encode the target variable.
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)
print('Encoding mapping:', le.classes_)

# Identify rare classes that need oversampling.
rare_classes = [
    label for label, count in pd.Series(y_train).value_counts().items()
    if count < 0.05 * len(y_train)
]
print("Rare classes:")
for cls in rare_classes:
    print(f"  {cls}: {le.classes_[cls]}")

# Define column groups (this example will one-hot encode all columns, so cat_cols and num_cols are not used in the transformer).
cat_cols = {'intake_type', 'intake_condition', 'animal_type', 'sex_upon_intake', 'breed', 'intake_hour', 'intake_year', 'primary_color', 'season', 'is_mix', 'age_group'}
categorical_features = [col for col in X_train.columns if col in cat_cols]


'''
# cat_cols_freq = ['primary_color']

# Frequency encode selected high-cardinality features
for col in cat_cols_freq:
  freq_map = X_train[col].value_counts()
  X_train[col] = X_train[col].map(freq_map)
  X_test[col]  = X_test[col].map(freq_map).fillna(0)
  '''

# Train the classifier with the refactored pipeline.


best_model, test_predictions = train_logreg_classifier(
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
   # rare_classes=rare_classes
)

# Get predicted probabilities from the best model
#y_val_proba = best_model.predict_proba(X_test)

# Define per-class thresholds (example: tune these)
# Order: [Adoption, Died, Euthanasia, RTO, Transfer]
thresholds = [0.6, 0.3, 0.3, 0.4, 0.4]


# Apply thresholds to the predicted probabilities
# thresholded_preds = apply_thresholds(y_val_proba, thresholds)

# Convert predictions back to original labels.
classification_report_with_accuracy_score(y_test, test_predictions)
#print(importance)

Encoding mapping: ['Adoption' 'Died' 'Euthanasia' 'Return to Owner' 'Transfer']
Rare classes:
  2: Euthanasia
  1: Died
Fitting 5 folds for each of 10 candidates, totalling 50 fits


KeyboardInterrupt: 

In [ ]:
# Bagging attempt\
# bagging logreg
def train_bagging_classifier(X_train, y_train, X_test, base_model, n_estimators=50):
    """
    Trains a Bagging Classifier with the given base model and evaluates it on the test data.
    
    Parameters:
    - X_train: Training features
    - y_train: Training labels
    - X_test: Test features
    - base_model: The base model to use for the bagging (e.g., Logistic Regression)
    - n_estimators: Number of base models in the ensemble (default is 50)
    
    Returns:
    - trained_model: Trained BaggingClassifier
    - test_predictions: Predictions on the test set
    """
    # Create the BaggingClassifier with the base model
    bagging_clf = BaggingClassifier(base_estimator=base_model, n_estimators=n_estimators, random_state=42, n_jobs=-1)
    
    # Fit the model
    bagging_clf.fit(X_train, y_train)
    
    # Make predictions
    test_predictions = bagging_clf.predict(X_test)
    
    # Evaluate the model
    training_accuracy = balanced_accuracy_score(y_train, bagging_clf.predict(X_train))
    print(f"Training Accuracy: {training_accuracy:.4f}")

    # needs CV if not a flop
    
    return bagging_clf, test_predictions


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

# Create a base Logistic Regression model
logreg, test_predict= train_logreg_classifier(X_train=X_train, y_train=y_train, X_test=X_test,)

bagging_model, test_predictions = train_bagging_classifier(X_train, y_train, X_test, base_model=logreg, n_estimators=50)

classification_report_with_accuracy_score(y_test, test_predictions)